<a href="https://colab.research.google.com/github/alexanderknysh/adcpml/blob/main/adcpml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

url = 'https://github.com/alexanderknysh/adcpml/blob/main/common_data.xlsx?raw=true'

# upload velocity dataset: west and east adpc profiles
general = pd.read_excel(url)

In [5]:
display(general)

,Date,WaterDepth,WaveHeight,WavePeriod
0,2019-05-16 14:15:00.000,9.475,0.235,11.075
1,2019-05-16 14:30:00.000,9.395,0.250,12.150
2,2019-05-16 14:44:59.990,9.289,0.240,11.370
3,2019-05-16 14:59:59.985,9.160,0.235,11.555
4,2019-05-16 15:14:59.980,9.007,0.210,11.640
...,...,...,...,...
1157,2019-05-28 15:29:54.215,7.275,0.275,8.780
1158,2019-05-28 15:44:54.210,7.176,0.260,8.640
1159,2019-05-28 15:59:54.205,7.083,0.260,8.660
1160,2019-05-28 16:14:54.200,7.040,0.280,8.360
